-------------------------------------

### Neural Networking

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
from pathlib import Path



In [24]:
# Import our input dataset
data = Path('Resources_k/fire_size_bins_new.csv')
df = pd.read_csv(data)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Resources_k\\fire_size_bins_new.csv'

In [3]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "year", "putout_time"],inplace=True)

In [4]:
df

,fire_cause,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin
0,Debris Burning,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup
1,Arson,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini
2,Debris Burning,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup
3,Debris Burning,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini
4,Lightning,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,Utilities,TX,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large
13134,Utilities,TX,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large
13135,Accidental,OR,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL
13136,Debris Burning,MT,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large


In [5]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=['fire_cause', 'discovery_month', 'state']

# Check the number of unique values in each column
df[cat].nunique()



fire_cause          5
discovery_month    12
state              45
dtype: int64

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities,discovery_month_Apr,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,...,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,state_WV,state_WY
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,state_WV,state_WY
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df.columns

Index(['Temp_pre_30', 'Temp_pre_15', 'Temp_pre_7', 'Wind_pre_30',
       'Wind_pre_15', 'Wind_pre_7', 'Hum_pre_30', 'Hum_pre_15', 'Hum_pre_7',
       'fire_size_bin', 'fire_cause_Accidental', 'fire_cause_Arson',
       'fire_cause_Debris Burning', 'fire_cause_Lightning',
       'fire_cause_Utilities', 'discovery_month_Apr', 'discovery_month_Aug',
       'discovery_month_Dec', 'discovery_month_Feb', 'discovery_month_Jan',
       'discovery_month_Jul', 'discovery_month_Jun', 'discovery_month_Mar',
       'discovery_month_May', 'discovery_month_Nov', 'discovery_month_Oct',
       'discovery_month_Sep', 'state_AK', 'state_AL', 'state_AR', 'state_AZ',
       'state_CA', 'state_CO', 'state_FL', 'state_GA', 'state_IA', 'state_ID',
       'state_IL', 'state_IN', 'state_KS', 'state_KY', 'state_LA', 'state_MA',
       'state_MD', 'state_ME', 'state_MI', 'state_MN', 'state_MO', 'state_MS',
       'state_MT', 'state_NC', 'state_ND', 'state_NE', 'state_NJ', 'state_NM',
       'state_NV', 'state_NY'

In [9]:
# # Import label encoder
# from sklearn import preprocessing
 
# # label_encoder object knows how to understand word labels.
# label_encoder = preprocessing.LabelEncoder()
 
# # Encode labels in column 'fire_size_bin.
# df['fire_size_bin']= label_encoder.fit_transform(df['fire_size_bin'])
 
# df['fire_size_bin'].unique()

In [10]:
df["size_bin_no"]=''
df

,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,state_WV,state_WY,size_bin_no
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [11]:
df.loc[(df['fire_size_bin']=="Teacup"),"size_bin_no"] = 1
df.loc[(df['fire_size_bin']=="Toy"),"size_bin_no"] = 2
df.loc[(df['fire_size_bin']=="Mini"),"size_bin_no"] = 3
df.loc[(df['fire_size_bin']=="Medium"),"size_bin_no"] = 4
df.loc[(df['fire_size_bin']=="Large"),"size_bin_no"] = 5
df.loc[(df['fire_size_bin']=="XL"),"size_bin_no"] = 6
df


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,state_WV,state_WY,size_bin_no
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [12]:
df['size_bin_no']=df.size_bin_no.astype(int)

In [13]:
df.drop(['fire_size_bin'], axis=1, inplace=True)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13138 entries, 0 to 13137
Data columns (total 72 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Temp_pre_30                13138 non-null  float64
 1   Temp_pre_15                13138 non-null  float64
 2   Temp_pre_7                 13138 non-null  float64
 3   Wind_pre_30                13138 non-null  float64
 4   Wind_pre_15                13138 non-null  float64
 5   Wind_pre_7                 13138 non-null  float64
 6   Hum_pre_30                 13138 non-null  float64
 7   Hum_pre_15                 13138 non-null  float64
 8   Hum_pre_7                  13138 non-null  float64
 9   fire_cause_Accidental      13138 non-null  float64
 10  fire_cause_Arson           13138 non-null  float64
 11  fire_cause_Debris Burning  13138 non-null  float64
 12  fire_cause_Lightning       13138 non-null  float64
 13  fire_cause_Utilities       13138 non-null  flo

In [15]:
df.columns

Index(['Temp_pre_30', 'Temp_pre_15', 'Temp_pre_7', 'Wind_pre_30',
       'Wind_pre_15', 'Wind_pre_7', 'Hum_pre_30', 'Hum_pre_15', 'Hum_pre_7',
       'fire_cause_Accidental', 'fire_cause_Arson',
       'fire_cause_Debris Burning', 'fire_cause_Lightning',
       'fire_cause_Utilities', 'discovery_month_Apr', 'discovery_month_Aug',
       'discovery_month_Dec', 'discovery_month_Feb', 'discovery_month_Jan',
       'discovery_month_Jul', 'discovery_month_Jun', 'discovery_month_Mar',
       'discovery_month_May', 'discovery_month_Nov', 'discovery_month_Oct',
       'discovery_month_Sep', 'state_AK', 'state_AL', 'state_AR', 'state_AZ',
       'state_CA', 'state_CO', 'state_FL', 'state_GA', 'state_IA', 'state_ID',
       'state_IL', 'state_IN', 'state_KS', 'state_KY', 'state_LA', 'state_MA',
       'state_MD', 'state_ME', 'state_MI', 'state_MN', 'state_MO', 'state_MS',
       'state_MT', 'state_NC', 'state_ND', 'state_NE', 'state_NJ', 'state_NM',
       'state_NV', 'state_NY', 'state_OH', 'st

In [16]:
# Remove loan status target from features data
target=df.size_bin_no
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=["size_bin_no", ]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 140
hidden_nodes_layer2 = 70
# hidden_nodes_layer3 = 11


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 140)               10080     
                                                                 
 dense_1 (Dense)             (None, 70)                9870      
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
Total params: 20,021
Trainable params: 20,021
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [19]:
## Import checkpoint dependencies
# import os
# from tensorflow.keras.callbacks import ModelCheckpoint

# # Define the checkpoint path and filenames
# os.makedirs("checkpoints/",exist_ok=True)
# checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# # Create a callback that saves the model's weights every 5 epoch
# cp_callback = ModelCheckpoint(
#     filepath=checkpoint_path,
#     verbose=5,
#     save_weights_only=True,
#     save_freq='epoch')

# Train the model
# fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# # Restore the model weights
# nn.load_weights("checkpoints/weights.100.hdf5")


Epoch 1/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4491
Epoch 2/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 3/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 4/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 5/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 6/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 7/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 8/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 9/50
308/308 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4495
Epoch 10/50
308/308 [==============================] - 1s 2ms/st

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

103/103 - 0s - loss: 0.0000e+00 - accuracy: 0.4645 - 309ms/epoch - 3ms/step
Loss: 0.0, Accuracy: 0.46453577280044556


---------------------

------------------------------

### Random Forest Classifier

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from pathlib import Path
from imblearn.ensemble import BalancedRandomForestClassifier

# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()

,Unnamed: 0,fire_id,fire_size,fire_cause,latitude,longitude,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,year,putout_time,fire_size_bin
0,0,3,1.00,Debris Burning,39.641400,-119.308300,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,2005,0,Teacup
1,1,24,40.00,Arson,31.435181,-88.999489,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1999,0,Mini
2,3,31,1.20,Debris Burning,48.833000,-99.783600,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,2007,0,Teacup
3,4,35,30.18,Debris Burning,31.259000,-84.895600,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,2000,0,Mini
4,5,36,1420.00,Lightning,33.241800,-104.912200,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1994,1,Large


In [7]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "year", "putout_time", "fire_cause"],inplace=True)
df

,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin
0,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup
1,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini
2,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup
3,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini
4,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large
...,...,...,...,...,...,...,...,...,...,...,...,...
13133,TX,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large
13134,TX,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large
13135,OR,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL
13136,MT,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large


In [8]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=["discovery_month", "state"]

# Check the number of unique values in each column
df[cat].nunique()

discovery_month    12
state              45
dtype: int64

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,discovery_month_Apr,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,...,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,state_WV,state_WY
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,state_WV,state_WY
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Remove loan status target from features data
target=df.fire_size_bin
y=target
#X = df.loc[:, df.columns !=target]
X=df.copy()
X=df.drop(columns=['fire_size_bin'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
X

,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,discovery_month_Apr,...,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,state_WV,state_WY
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=78) 

#Fitting the model
model = model.fit(X_train, y_train)

# Making predictions using the testing data.
y_pred = model.predict(X_test)

In [16]:
# Calculating the accuracy score.
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.4998477929984779

In [17]:
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,135,9,12,152,6,95
Actual Toy,32,19,26,179,8,6
Actual Mini,33,13,47,329,11,12
Actual Medium,77,21,83,1295,18,32
Actual Large,14,9,30,253,14,9
Actual XL,84,3,3,82,2,132


In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

      Large       0.36      0.33      0.92      0.34      0.55      0.28       409
     Medium       0.26      0.07      0.98      0.11      0.26      0.06       270
       Mini       0.23      0.11      0.95      0.15      0.32      0.09       445
     Teacup       0.57      0.85      0.43      0.68      0.61      0.38      1526
        Toy       0.24      0.04      0.98      0.07      0.20      0.04       329
         XL       0.46      0.43      0.95      0.45      0.64      0.39       306

avg / total       0.43      0.50      0.71      0.44      0.50      0.27      3285



In [29]:
# Displaying Easy Ensemble Classifier
print("Random Forest Clasifier")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Random Forest Clasifier
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,128,8,10,182,4,77
Actual Toy,33,9,17,198,8,5
Actual Mini,38,8,27,357,8,7
Actual Medium,109,8,55,1297,18,39
Actual Large,20,5,22,266,10,6
Actual XL,81,2,4,109,2,108


Accuracy Score : 0.4806697108066971
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.31      0.31      0.90      0.31      0.53      0.27       409
     Medium       0.23      0.03      0.99      0.06      0.18      0.03       270
       Mini       0.20      0.06      0.96      0.09      0.24      0.05       445
     Teacup       0.54      0.85      0.37      0.66      0.56      0.33      1526
        Toy       0.20      0.03      0.99      0.05      0.17      0.03       329
         XL       0.45      0.35      0.96      0.39      0.58      0.32       306

avg / total       0.40      0.48      0.68      0.40      0.44      0.23      3285



In [30]:
## CCaallccuullaattee ffeeaattuurree iimmppoorrttaannccee iinn tthhee RRaannddoomm FFoorreesstt mmooddeell..
importances = model.feature_importances_
importances

array([0.10897481, 0.10746183, 0.10966604, 0.10599194, 0.10251427,
       0.10490513, 0.12576574, 0.11642294, 0.11829731])

In [31]:
# We can sort the features by their importance.
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.12576573744694397, 'Hum_pre_30'),
 (0.11829730769544074, 'Hum_pre_7'),
 (0.11642293872883432, 'Hum_pre_15'),
 (0.10966603783894642, 'Temp_pre_7'),
 (0.10897481003756421, 'Temp_pre_30'),
 (0.10746183387405538, 'Temp_pre_15'),
 (0.10599193521860646, 'Wind_pre_30'),
 (0.1049051319266292, 'Wind_pre_7'),
 (0.10251426723297928, 'Wind_pre_15')]

### Random Forest Classifier

In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from pathlib import Path

# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()

,Unnamed: 0,fire_id,fire_size,fire_cause,latitude,longitude,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,year,putout_time,fire_size_bin
0,0,3,1.00,Debris Burning,39.641400,-119.308300,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,2005,0,Teacup
1,1,24,40.00,Arson,31.435181,-88.999489,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1999,0,Mini
2,3,31,1.20,Debris Burning,48.833000,-99.783600,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,2007,0,Teacup
3,4,35,30.18,Debris Burning,31.259000,-84.895600,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,2000,0,Mini
4,5,36,1420.00,Lightning,33.241800,-104.912200,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1994,1,Large


In [108]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "year", "putout_time"],inplace=True)
df

,fire_cause,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin
0,Debris Burning,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup
1,Arson,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini
2,Debris Burning,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup
3,Debris Burning,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini
4,Lightning,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,Utilities,TX,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large
13134,Utilities,TX,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large
13135,Accidental,OR,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL
13136,Debris Burning,MT,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large


In [109]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=["state", "discovery_month", "fire_cause"]

# Check the number of unique values in each column
df[cat].nunique()

state              45
discovery_month    12
fire_cause          5
dtype: int64

In [110]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,state_FL,state_GA,state_IA,state_ID,...,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [111]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [112]:
df["size_bin_no"]=''
df

,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities,size_bin_no
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,


In [113]:
df.loc[(df['fire_size_bin']=="Teacup"),"size_bin_no"] = 1
df.loc[(df['fire_size_bin']=="Toy"),"size_bin_no"] = 2
df.loc[(df['fire_size_bin']=="Mini"),"size_bin_no"] = 3
df.loc[(df['fire_size_bin']=="Medium"),"size_bin_no"] = 4
df.loc[(df['fire_size_bin']=="Large"),"size_bin_no"] = 5
df.loc[(df['fire_size_bin']=="XL"),"size_bin_no"] = 6
df


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities,size_bin_no
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,6
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5


In [114]:
df['size_bin_no']=df.size_bin_no.astype(int)
df.drop(columns=['fire_size_bin'], inplace=True)

In [115]:
# Remove loan status target from features data
target=df.size_bin_no
y=target
#X = df.loc[:, df.columns !=target]
X=df.copy()
X=df.drop(columns=['size_bin_no'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [116]:
X

,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,state_AK,...,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,fire_cause_Accidental,fire_cause_Arson,fire_cause_Debris Burning,fire_cause_Lightning,fire_cause_Utilities
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13133,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13134,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
13135,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
13136,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [117]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=78) 

#Fitting the model
model = model.fit(X_train, y_train)

# Making predictions using the testing data.
y_pred = model.predict(X_test)

In [118]:
# Calculating the accuracy score.
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.5074581430745815

In [119]:
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,1322,19,74,8,75,28
Actual Toy,263,8,27,5,17,9
Actual Mini,337,6,46,12,34,10
Actual Medium,185,8,27,14,27,9
Actual Large,155,7,11,6,139,91
Actual XL,77,2,3,1,85,138


In [106]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

          1       0.58      0.85      0.45      0.69      0.62      0.40      1526
          2       0.23      0.04      0.99      0.06      0.19      0.03       329
          3       0.27      0.13      0.94      0.18      0.35      0.12       445
          4       0.19      0.05      0.98      0.08      0.23      0.05       270
          5       0.38      0.37      0.92      0.38      0.58      0.32       409
          6       0.46      0.43      0.95      0.44      0.64      0.38       306

avg / total       0.43      0.51      0.72      0.44      0.51      0.29      3285



In [79]:
# Displaying Easy Ensemble Classifier
print("Random Forest Clasifier")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Random Forest Clasifier
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,1302,18,82,18,74,32
Actual Toy,255,14,29,8,14,9
Actual Mini,330,11,46,13,33,12
Actual Medium,180,8,26,18,32,6
Actual Large,153,6,12,9,134,95
Actual XL,82,2,3,3,84,132


Accuracy Score : 0.5010654490106545
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          1       0.57      0.85      0.43      0.68      0.61      0.38      1526
          2       0.24      0.04      0.98      0.07      0.20      0.04       329
          3       0.23      0.10      0.95      0.14      0.31      0.09       445
          4       0.26      0.07      0.98      0.11      0.26      0.06       270
          5       0.36      0.33      0.92      0.34      0.55      0.28       409
          6       0.46      0.43      0.95      0.45      0.64      0.39       306

avg / total       0.43      0.50      0.71      0.44      0.49      0.27      3285



In [80]:
## CCaallccuullaattee ffeeaattuurree iimmppoorrttaannccee iinn tthhee RRaannddoomm FFoorreesstt mmooddeell..
importances = model.feature_importances_
importances

array([8.79341355e-02, 8.72537961e-02, 8.74450021e-02, 8.41548326e-02,
       8.31543527e-02, 8.36113137e-02, 9.73494696e-02, 9.53652471e-02,
       9.72190478e-02, 6.34773640e-03, 7.40708783e-03, 2.03437452e-03,
       3.85100878e-03, 1.65228522e-03, 7.68332347e-03, 9.12115524e-03,
       6.51957522e-03, 5.27953975e-03, 3.63668820e-03, 4.60193160e-03,
       5.42687265e-03, 2.15113515e-02, 4.31686828e-03, 2.61511907e-03,
       4.90671903e-03, 5.56439571e-03, 2.31596203e-03, 2.58833358e-03,
       6.91637641e-03, 5.55515003e-05, 6.61397282e-03, 5.03921678e-04,
       4.40591327e-04, 1.10576793e-03, 2.73708422e-03, 5.50806990e-04,
       4.83623445e-06, 1.95104184e-04, 3.45295575e-04, 1.05941864e-03,
       2.17742972e-03, 1.46465204e-03, 4.73725569e-03, 3.97097654e-03,
       2.04325490e-03, 1.69060654e-03, 2.09340764e-03, 2.14937807e-04,
       3.89459756e-03, 4.04050049e-03, 6.96763787e-03, 4.48161255e-04,
       6.00220661e-03, 3.20970433e-03, 7.08979807e-04, 5.30001645e-05,
      

In [64]:
# We can sort the features by their importance.
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.09734946962964271, 'Hum_pre_30'),
 (0.09721904783368264, 'Hum_pre_7'),
 (0.0953652471209638, 'Hum_pre_15'),
 (0.0879341354735827, 'Temp_pre_30'),
 (0.08744500212784666, 'Temp_pre_7'),
 (0.08725379614202776, 'Temp_pre_15'),
 (0.08415483264604201, 'Wind_pre_30'),
 (0.08361131372711372, 'Wind_pre_7'),
 (0.08315435270824707, 'Wind_pre_15'),
 (0.021511351500400986, 'state_AK'),
 (0.00912115524455081, 'discovery_month_Jun'),
 (0.007683323472355947, 'discovery_month_Jul'),
 (0.0074070878271044445, 'discovery_month_Aug'),
 (0.0069676378676676995, 'state_NY'),
 (0.006916376408601078, 'state_GA'),
 (0.006613972819465441, 'state_ID'),
 (0.006519575222277322, 'discovery_month_Mar'),
 (0.006347736403758375, 'discovery_month_Apr'),
 (0.006002206614099295, 'state_OK'),
 (0.005564395710539236, 'state_CA'),
 (0.005426872652731194, 'discovery_month_Sep'),
 (0.005279539754899029, 'discovery_month_May'),
 (0.004906719031606131, 'state_AZ'),
 (0.004737255685218033, 'state_MS'),
 (0.0046019315979311005,

-------------------------

----------------------------------

### Easy Ensemble Classifier

In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from pathlib import Path

# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()

,Unnamed: 0,fire_id,fire_size,fire_cause,latitude,longitude,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,year,putout_time,fire_size_bin
0,0,3,1.00,Debris Burning,39.641400,-119.308300,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,2005,0,Teacup
1,1,24,40.00,Arson,31.435181,-88.999489,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1999,0,Mini
2,3,31,1.20,Debris Burning,48.833000,-99.783600,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,2007,0,Teacup
3,4,35,30.18,Debris Burning,31.259000,-84.895600,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,2000,0,Mini
4,5,36,1420.00,Lightning,33.241800,-104.912200,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1994,1,Large


In [45]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "year", "putout_time", "state", "fire_cause"],inplace=True)
df

,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin
0,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup
1,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini
2,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup
3,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini
4,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large
...,...,...,...,...,...,...,...,...,...,...,...
13133,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large
13134,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large
13135,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL
13136,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large


In [46]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=["discovery_month"]

# Check the number of unique values in each column
df[cat].nunique()

discovery_month    12
dtype: int64

In [47]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,discovery_month_Apr,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Remove loan status target from features data
target=df.fire_size_bin
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=['fire_size_bin']).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: could not convert string to float: 'May'

In [52]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
model = EasyEnsembleClassifier(n_estimators=100, random_state=1) 

#Fitting the model
model = model.fit(X_train, y_train)

# Making predictions using the testing data.
y_pred = model.predict(X_test)

In [55]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.30532724505327247

In [56]:
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,159,21,8,26,15,173
Actual Toy,59,31,69,47,35,42
Actual Mini,63,42,166,94,67,45
Actual Medium,166,131,383,430,193,189
Actual Large,28,24,97,86,44,37
Actual XL,100,11,3,26,2,173


In [57]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      Large       0.28      0.40      0.86      0.33      0.58      0.32       402
     Medium       0.12      0.11      0.92      0.11      0.32      0.09       283
       Mini       0.23      0.35      0.80      0.28      0.53      0.27       477
     Teacup       0.61      0.29      0.84      0.39      0.49      0.23      1492
        Toy       0.12      0.14      0.89      0.13      0.35      0.12       316
         XL       0.26      0.55      0.84      0.36      0.68      0.45       315

avg / total       0.39      0.31      0.85      0.31      0.50      0.24      3285



In [58]:
# Displaying Easy Ensemble Classifier
print("Balanced Random Forest Clasifier")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Balanced Random Forest Clasifier
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,159,21,8,26,15,173
Actual Toy,59,31,69,47,35,42
Actual Mini,63,42,166,94,67,45
Actual Medium,166,131,383,430,193,189
Actual Large,28,24,97,86,44,37
Actual XL,100,11,3,26,2,173


Accuracy Score : 0.30532724505327247
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.28      0.40      0.86      0.33      0.58      0.32       402
     Medium       0.12      0.11      0.92      0.11      0.32      0.09       283
       Mini       0.23      0.35      0.80      0.28      0.53      0.27       477
     Teacup       0.61      0.29      0.84      0.39      0.49      0.23      1492
        Toy       0.12      0.14      0.89      0.13      0.35      0.12       316
         XL       0.26      0.55      0.84      0.36      0.68      0.45       315

avg / total       0.39      0.31      0.85      0.31      0.50      0.24      3285



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from pathlib import Path

# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "year", "putout_time", "state", "fire_cause"],inplace=True)
df
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=["discovery_month"]

# Check the number of unique values in each column
df[cat].nunique()
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()
# Remove loan status target from features data
target=df.fire_size_bin
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=['fire_size_bin']).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [2]:

### Balanced Random Forest Classifier
# Resample the training data with the BalancedRandomForestClassifier

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 

#Fitting the model
model = model.fit(X_train, y_train)

# Making predictions using the testing data.
y_pred = model.predict(X_test)
# Calculating the accuracy score.
from sklearn.metrics import balanced_accuracy_score
acc_score = balanced_accuracy_score(y_test, y_pred)

from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

# Displaying Easy Ensemble Classifier
print("Balanced Random Forest Clasifier")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)


                   pre       rec       spe        f1       geo       iba       sup

      Large       0.30      0.45      0.85      0.36      0.62      0.37       402
     Medium       0.14      0.21      0.87      0.17      0.43      0.17       283
       Mini       0.20      0.28      0.82      0.23      0.47      0.21       477
     Teacup       0.64      0.24      0.89      0.35      0.46      0.20      1492
        Toy       0.12      0.18      0.85      0.14      0.40      0.15       316
         XL       0.32      0.53      0.88      0.40      0.68      0.45       315

avg / total       0.41      0.29      0.87      0.30      0.49      0.24      3285

Balanced Random Forest Clasifier
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,180,32,9,17,19,145
Actual Toy,51,60,65,35,41,31
Actual Mini,59,66,132,87,95,38
Actual Medium,176,210,354,360,269,123
Actual Large,36,54,87,60,58,21
Actual XL,107,18,4,6,14,166


Accuracy Score : 0.31472003035104495
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.30      0.45      0.85      0.36      0.62      0.37       402
     Medium       0.14      0.21      0.87      0.17      0.43      0.17       283
       Mini       0.20      0.28      0.82      0.23      0.47      0.21       477
     Teacup       0.64      0.24      0.89      0.35      0.46      0.20      1492
        Toy       0.12      0.18      0.85      0.14      0.40      0.15       316
         XL       0.32      0.53      0.88      0.40      0.68      0.45       315

avg / total       0.41      0.29      0.87      0.30      0.49      0.24      3285



AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [9]:
# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()

,Unnamed: 0,fire_id,fire_size,fire_cause,latitude,longitude,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,year,putout_time,fire_size_bin
0,0,3,1.00,Debris Burning,39.641400,-119.308300,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,2005,0,Teacup
1,1,24,40.00,Arson,31.435181,-88.999489,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1999,0,Mini
2,3,31,1.20,Debris Burning,48.833000,-99.783600,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,2007,0,Teacup
3,4,35,30.18,Debris Burning,31.259000,-84.895600,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,2000,0,Mini
4,5,36,1420.00,Lightning,33.241800,-104.912200,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1994,1,Large


In [10]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "year", "putout_time", "state", "fire_cause"],inplace=True)
df

,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin
0,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup
1,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini
2,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup
3,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini
4,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large
...,...,...,...,...,...,...,...,...,...,...,...
13133,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large
13134,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large
13135,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL
13136,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large


In [11]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=["discovery_month"]

# Check the number of unique values in each column
df[cat].nunique()

discovery_month    12
dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,discovery_month_Apr,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Remove loan status target from features data
target=df.fire_size_bin
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=['fire_size_bin']).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
import warnings
warnings.filterwarnings('ignore')

from collections import Counter

# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'Toy': 4463,
         'Teacup': 4463,
         'XL': 4463,
         'Mini': 4463,
         'Medium': 4463,
         'Large': 4463})

In [17]:
# Logistic regression using random oversampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
acc_score= balanced_accuracy_score(y_test, y_pred)

In [20]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,160,38,4,39,38,123
Actual Toy,58,56,42,47,48,32
Actual Mini,59,72,98,82,98,68
Actual Medium,171,193,290,303,324,211
Actual Large,27,38,74,58,78,41
Actual XL,111,24,1,29,26,124


In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

      Large       0.27      0.40      0.85      0.32      0.58      0.32       402
     Medium       0.13      0.20      0.88      0.16      0.42      0.16       283
       Mini       0.19      0.21      0.85      0.20      0.42      0.16       477
     Teacup       0.54      0.20      0.86      0.30      0.42      0.16      1492
        Toy       0.13      0.25      0.82      0.17      0.45      0.19       316
         XL       0.21      0.39      0.84      0.27      0.58      0.32       315

avg / total       0.35      0.25      0.85      0.26      0.46      0.20      3285



In [22]:
# Displaying Random Oversampling results
print("Naive Random Oversampling")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Naive Random Oversampling
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,160,38,4,39,38,123
Actual Toy,58,56,42,47,48,32
Actual Mini,59,72,98,82,98,68
Actual Medium,171,193,290,303,324,211
Actual Large,27,38,74,58,78,41
Actual XL,111,24,1,29,26,124


Accuracy Score : 0.27415164821116
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.27      0.40      0.85      0.32      0.58      0.32       402
     Medium       0.13      0.20      0.88      0.16      0.42      0.16       283
       Mini       0.19      0.21      0.85      0.20      0.42      0.16       477
     Teacup       0.54      0.20      0.86      0.30      0.42      0.16      1492
        Toy       0.13      0.25      0.82      0.17      0.45      0.19       316
         XL       0.21      0.39      0.84      0.27      0.58      0.32       315

avg / total       0.35      0.25      0.85      0.26      0.46      0.20      3285



### SMOTE Oversampling

In [23]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'Toy': 4463,
         'Teacup': 4463,
         'XL': 4463,
         'Mini': 4463,
         'Medium': 4463,
         'Large': 4463})

In [24]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
acc_score=balanced_accuracy_score(y_test, y_pred)

In [27]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,149,27,20,29,40,137
Actual Toy,58,29,80,29,46,41
Actual Mini,56,38,153,51,98,81
Actual Medium,163,103,467,215,294,250
Actual Large,28,21,115,41,68,43
Actual XL,108,16,10,24,27,130


In [28]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      Large       0.27      0.37      0.86      0.31      0.56      0.30       402
     Medium       0.12      0.10      0.93      0.11      0.31      0.09       283
       Mini       0.18      0.32      0.75      0.23      0.49      0.23       477
     Teacup       0.55      0.14      0.90      0.23      0.36      0.12      1492
        Toy       0.12      0.22      0.83      0.15      0.42      0.17       316
         XL       0.19      0.41      0.81      0.26      0.58      0.32       315

avg / total       0.35      0.23      0.86      0.22      0.43      0.18      3285



In [29]:
# Displaying SMOTE results
print("SMOTE Oversampling")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,149,27,20,29,40,137
Actual Toy,58,29,80,29,46,41
Actual Mini,56,38,153,51,98,81
Actual Medium,163,103,467,215,294,250
Actual Large,28,21,115,41,68,43
Actual XL,108,16,10,24,27,130


Accuracy Score : 0.26097752402920654
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.27      0.37      0.86      0.31      0.56      0.30       402
     Medium       0.12      0.10      0.93      0.11      0.31      0.09       283
       Mini       0.18      0.32      0.75      0.23      0.49      0.23       477
     Teacup       0.55      0.14      0.90      0.23      0.36      0.12      1492
        Toy       0.12      0.22      0.83      0.15      0.42      0.17       316
         XL       0.19      0.41      0.81      0.26      0.58      0.32       315

avg / total       0.35      0.23      0.86      0.22      0.43      0.18      3285



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)


Counter({'Large': 732,
         'Medium': 732,
         'Mini': 732,
         'Teacup': 732,
         'Toy': 732,
         'XL': 732})

In [31]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
acc_score= balanced_accuracy_score(y_test, y_pred)

In [35]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,99,22,1,27,63,190
Actual Toy,46,32,15,49,89,52
Actual Mini,41,45,41,63,188,99
Actual Medium,115,97,97,311,575,297
Actual Large,22,17,32,69,124,52
Actual XL,71,24,4,14,37,165


In [36]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      Large       0.25      0.25      0.90      0.25      0.47      0.21       402
     Medium       0.14      0.11      0.93      0.12      0.32      0.10       283
       Mini       0.22      0.09      0.95      0.12      0.29      0.07       477
     Teacup       0.58      0.21      0.88      0.31      0.43      0.17      1492
        Toy       0.12      0.39      0.68      0.18      0.52      0.26       316
         XL       0.19      0.52      0.77      0.28      0.63      0.39       315

avg / total       0.37      0.24      0.86      0.24      0.43      0.18      3285



In [37]:
# Displaying Undersampling results
print("Cluster Centroid Undersampling")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Cluster Centroid Undersampling
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,99,22,1,27,63,190
Actual Toy,46,32,15,49,89,52
Actual Mini,41,45,41,63,188,99
Actual Medium,115,97,97,311,575,297
Actual Large,22,17,32,69,124,52
Actual XL,71,24,4,14,37,165


Accuracy Score : 0.2616593945642105
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.25      0.25      0.90      0.25      0.47      0.21       402
     Medium       0.14      0.11      0.93      0.12      0.32      0.10       283
       Mini       0.22      0.09      0.95      0.12      0.29      0.07       477
     Teacup       0.58      0.21      0.88      0.31      0.43      0.17      1492
        Toy       0.12      0.39      0.68      0.18      0.52      0.26       316
         XL       0.19      0.52      0.77      0.28      0.63      0.39       315

avg / total       0.37      0.24      0.86      0.24      0.43      0.18      3285



-----------------------------------------------------

# Deliverable 2

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [38]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'Large': 4122,
         'Medium': 4701,
         'Mini': 3564,
         'Teacup': 497,
         'Toy': 4368,
         'XL': 4536})

In [39]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [40]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)

In [41]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
cm_df = pd.DataFrame(
    cm, index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,99,22,1,27,63,190
Actual Toy,46,32,15,49,89,52
Actual Mini,41,45,41,63,188,99
Actual Medium,115,97,97,311,575,297
Actual Large,22,17,32,69,124,52
Actual XL,71,24,4,14,37,165


In [42]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      Large       0.26      0.32      0.87      0.29      0.53      0.27       402
     Medium       0.12      0.28      0.81      0.17      0.48      0.22       283
       Mini       0.24      0.03      0.99      0.05      0.16      0.02       477
     Teacup       0.00      0.00      1.00      0.00      0.00      0.00      1492
        Toy       0.13      0.56      0.60      0.21      0.58      0.33       316
         XL       0.19      0.43      0.80      0.26      0.59      0.33       315

avg / total       0.11      0.16      0.91      0.10      0.24      0.12      3285



In [43]:
# Displaying SMOTEEN results
print("SMOTEENN Sampling")
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

SMOTEENN Sampling
Confusion Matrix


,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,99,22,1,27,63,190
Actual Toy,46,32,15,49,89,52
Actual Mini,41,45,41,63,188,99
Actual Medium,115,97,97,311,575,297
Actual Large,22,17,32,69,124,52
Actual XL,71,24,4,14,37,165


Accuracy Score : 0.269398378599054
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

      Large       0.26      0.32      0.87      0.29      0.53      0.27       402
     Medium       0.12      0.28      0.81      0.17      0.48      0.22       283
       Mini       0.24      0.03      0.99      0.05      0.16      0.02       477
     Teacup       0.00      0.00      1.00      0.00      0.00      0.00      1492
        Toy       0.13      0.56      0.60      0.21      0.58      0.33       316
         XL       0.19      0.43      0.80      0.26      0.59      0.33       315

avg / total       0.11      0.16      0.91      0.10      0.24      0.12      3285



In [8]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


In [9]:
# Import our input dataset
data = Path('Resources_k/fire_size_bins.csv')
df = pd.read_csv(data)
df.head()

,Unnamed: 0,fire_id,fire_size,fire_cause,latitude,longitude,state,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,year,putout_time,fire_size_bin
0,0,3,1.00,Debris Burning,39.641400,-119.308300,NV,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,2005,0,Teacup
1,1,24,40.00,Arson,31.435181,-88.999489,MS,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,1999,0,Mini
2,3,31,1.20,Debris Burning,48.833000,-99.783600,ND,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,2007,0,Teacup
3,4,35,30.18,Debris Burning,31.259000,-84.895600,GA,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,2000,0,Mini
4,5,36,1420.00,Lightning,33.241800,-104.912200,NM,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,1994,1,Large


In [10]:
df.drop(columns=["Unnamed: 0", "fire_id", "fire_size", "latitude", "longitude", "year", "putout_time", "state", "fire_cause"],inplace=True)
df

,discovery_month,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin
0,Jun,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup
1,Apr,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini
2,Apr,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup
3,Oct,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini
4,Jul,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large
...,...,...,...,...,...,...,...,...,...,...,...
13133,Aug,30.731860,30.545367,28.719917,3.136761,3.238803,3.224274,50.089157,49.416505,55.426471,Large
13134,Oct,24.642268,23.713390,24.221869,1.529850,1.576828,1.563817,62.848172,60.641975,63.196819,Large
13135,Sep,15.546194,12.890633,10.734328,2.608150,2.486802,1.835821,55.009259,62.368700,67.266304,XL
13136,Mar,2.275974,7.360185,7.678571,4.428757,4.197593,3.803571,51.676681,46.933399,42.638384,Large


In [11]:
# Generate our categorical variable list
#cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat=["discovery_month"]

# Check the number of unique values in each column
df[cat].nunique()

discovery_month    12
dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat)
encode_df.head()

,discovery_month_Apr,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat,1)
df.head()

c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,fire_size_bin,...,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,16.275967,18.996181,18.142564,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,Teacup,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,13.468619,15.067227,15.604790,2.038268,1.737921,1.775904,57.997207,56.747191,59.614458,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.891635,0.372659,-4.273834,5.800667,6.012852,6.658621,77.575012,75.963981,71.173116,Teacup,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.079480,17.722714,18.188679,3.659840,3.366443,2.211429,67.551783,61.733788,60.328571,Mini,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,31.055859,32.523438,34.893333,4.026367,3.844922,3.695833,28.783203,25.789062,18.208333,Large,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Remove loan status target from features data
target=df.fire_size_bin
y=target
# X = df.loc[:, df.columns !=target]
X=df.drop(columns=['fire_size_bin']).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

  ### Choose best learning rate

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.456
Accuracy score (validation): 0.455

Learning rate:  0.1
Accuracy score (training): 0.483
Accuracy score (validation): 0.469

Learning rate:  0.25
Accuracy score (training): 0.511
Accuracy score (validation): 0.472

Learning rate:  0.5
Accuracy score (training): 0.529
Accuracy score (validation): 0.460

Learning rate:  0.75
Accuracy score (training): 0.540
Accuracy score (validation): 0.450

Learning rate:  1
Accuracy score (training): 0.541
Accuracy score (validation): 0.454



  ### Create Gradient Boosting Classifier

In [16]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.05,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).head(20)

,Prediction,Actual
4841,Teacup,Teacup
3257,Teacup,Teacup
3796,Teacup,Mini
5376,Teacup,Teacup
142,Teacup,Toy
3013,Teacup,Teacup
12112,Teacup,Large
6822,Teacup,Teacup
9282,Teacup,Teacup
7166,Teacup,Teacup


  ### Evaluate the model

In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.4550989345509893


In [20]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   index=["Actual Teacup", "Actual Toy", "Actual Mini", "Actual Medium", "Actual Large", "Actual XL"], columns=["Predicted Teacup", "Predicted Toy", "Predicted Mini", "Predicted Medium", "Predicted Large", "Predicted XL"]
)
cm_df

,Predicted Teacup,Predicted Toy,Predicted Mini,Predicted Medium,Predicted Large,Predicted XL
Actual Teacup,NaN,NaN,NaN,NaN,NaN,NaN
Actual Toy,NaN,NaN,NaN,NaN,NaN,NaN
Actual Mini,NaN,NaN,NaN,NaN,NaN,NaN
Actual Medium,NaN,NaN,NaN,NaN,NaN,NaN
Actual Large,NaN,NaN,NaN,NaN,NaN,NaN
Actual XL,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Generate classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

       Large       0.00      0.00      0.00       402
      Medium       0.00      0.00      0.00       283
        Mini       0.00      0.00      0.00       477
      Teacup       0.45      1.00      0.63      1492
         Toy       0.00      0.00      0.00       316
          XL       0.80      0.01      0.03       315

    accuracy                           0.46      3285
   macro avg       0.21      0.17      0.11      3285
weighted avg       0.28      0.46      0.29      3285



c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\KATLIN\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
